In [1]:
import polars as pl
import polars.selectors as cs

In [2]:
df = pl.read_ndjson('../data/ledgers/plata.ndjson')

In [3]:
from zlib import crc32

def assign_split(column: str) -> pl.Expr:

    seed = (
        pl.col(column)
        .cast(pl.String)
        .map_elements(lambda x: float(crc32(str.encode(x)) & 0xFFFFFFFF), return_dtype=pl.Float32)
        .mul(1 / 2**32)
    )

    return (
        pl.when(seed.is_between(0.0, 0.6))
        .then(pl.lit("train"))
        .when(seed.is_between(0.6, 0.8))
        .then(pl.lit("validate"))
        .when(seed.is_between(0.8, 1.0))
        .then(pl.lit("test"))
        .otherwise(pl.lit("train"))
    )

In [4]:
dynamic = (
    df.select(
        "bureau_search_ref",
        pl.col("created_at").str.replace("\+00", "").str.to_date(format="%Y-%m-%d %H:%M:%S%.f"),
        cs.by_dtype(pl.List(pl.String)) | cs.by_dtype(pl.List(pl.Int64)),
        target=pl.col("irb"),
    )
    .explode(pl.exclude("created_at","bureau_search_ref", "target"))
    .select(
        cs.all() - cs.string() - cs.by_name("account_index"),
        (cs.string() - cs.by_name("m")).replace("", None),
        statement_id=pl.concat_str("accgroupid", pl.lit("$"), "m"),
        account_open=pl.col("accstartdate").str.to_date(),
        account_index=pl.col("accgroupid").cast(pl.String),
        statement_date=pl.col("m").str.to_date(format="%Y-%m"),
    )
    # filter out statements of accounts with empty balances
    .filter(pl.col("bal") > 0)
    .select(
        "payamt", "bal", "history_limit", "bureau_search_ref", "pay", "acctypecode", "target", "statement_id", "account_index",
        # calculate two duration fields (time diff between statement and application date, statement and account open date)
        statement_tenure=(pl.col("statement_date") - pl.col("created_at")).dt.total_days(),
        account_tenure=(pl.col("account_open") - pl.col("created_at")).dt.total_days(),
    )
    # .select(
    #     pl.exclude("payamt", "history_limit"),
    #     pl.col("payamt").fill_null(-1),
    #     pl.col("history_limit").fill_null(-1),
    # )
    .sort("bureau_search_ref", "statement_tenure")
    .group_by("bureau_search_ref")
    .agg(cs.all())
)

In [5]:
dynamic.describe()

statistic,bureau_search_ref,payamt,bal,history_limit,pay,acctypecode,target,statement_id,account_index,statement_tenure,account_tenure
str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""","""4542""",4542.0,4542.0,4542.0,4542.0,4542.0,4542.0,4542.0,4542.0,4542.0,4542.0
"""null_count""","""0""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",null,null,null,null,null,null,null,null,null,null,null
"""std""",null,null,null,null,null,null,null,null,null,null,null
"""min""","""BS43GGXB""",null,null,null,null,null,null,null,null,null,null
"""25%""",null,null,null,null,null,null,null,null,null,null,null
"""50%""",null,null,null,null,null,null,null,null,null,null,null
"""75%""",null,null,null,null,null,null,null,null,null,null,null
"""max""","""BS45AMVB""",null,null,null,null,null,null,null,null,null,null


In [6]:
static = (
    df.select(
        (cs.string() - cs.by_name('loan_account_ref')).replace("", None),
        # cs.integer().fill_null(-1),
        cs.integer(),
        split=assign_split('bureau_search_ref'),
    )
)

In [7]:
static.describe()

statistic,oq,irb,hrb,cbc,nq,oyb,bf,urb,created_at,nob,ibc,ubc,bureau_search_ref,an,total_balance_on_default_accoun4,seb,total_repayments_on_active_acco2,tic,ef,izb,yeb,as,drb,zd,total_balance_on_active_account1,ueb,hh,tg,em,ah,tp,ej,bn,nic,total_repayments_on_active_acco5,azb,…,lrb,fb,gauge2_score,ccc,teb,eb,ye,qic,oic,vm,gsb,ygc,st,tr,hn,wmb,vyb,mu,wm,pob,mhc,pn,sr,pic,jyb,peb,reb,xm,bh,xe,cs,xzb,rh,gg,mzb,fs,split
str,str,str,str,str,str,str,str,str,str,str,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str
"""count""","""4401""","""4542""","""4542""","""4123""","""4176""","""31""","""4542""","""4101""","""4542""","""3982""","""1177""","""4542""","""4542""",4410.0,4542.0,4542.0,4542.0,3997.0,4532.0,3388.0,4542.0,583.0,4526.0,307.0,4542.0,4542.0,1374.0,4542.0,1475.0,4542.0,4542.0,4542.0,4316.0,4542.0,4542.0,4113.0,…,4542.0,4542.0,4542.0,2181.0,4542.0,4540.0,4137.0,4542.0,4542.0,4508.0,4101.0,4542.0,4542.0,1278.0,4437.0,4094.0,884.0,727.0,4482.0,4463.0,4542.0,4192.0,1352.0,4542.0,3778.0,4542.0,4542.0,4350.0,4542.0,4033.0,774.0,4509.0,4542.0,4542.0,4255.0,286.0,"""4542"""
"""null_count""","""141""","""0""","""0""","""419""","""366""","""4511""","""0""","""441""","""0""","""560""","""3365""","""0""","""0""",132.0,0.0,0.0,0.0,545.0,10.0,1154.0,0.0,3959.0,16.0,4235.0,0.0,0.0,3168.0,0.0,3067.0,0.0,0.0,0.0,226.0,0.0,0.0,429.0,…,0.0,0.0,0.0,2361.0,0.0,2.0,405.0,0.0,0.0,34.0,441.0,0.0,0.0,3264.0,105.0,448.0,3658.0,3815.0,60.0,79.0,0.0,350.0,3190.0,0.0,764.0,0.0,0.0,192.0,0.0,509.0,3768.0,33.0,0.0,0.0,287.0,4256.0,"""0"""
"""mean""",null,null,null,null,null,null,null,null,null,null,null,null,null,122.934467,8.733598,2.353369,260.233377,72.482612,7.119153,10.804014,0.091369,1921.957118,12997.057667,1919.322476,13051.100837,4.245046,41.0,0.045575,138053.352542,1.32915,0.614927,0.369661,161.775023,0.056363,20.749229,3.170921,…,5.979304,11.325407,598.522017,34.552499,3.288639,22.905507,114.309161,0.3155,0.117349,111.043035,11.073884,0.27675,0.046455,47.934272,329.372324,61.228627,371.355204,23.887208,145.074743,162.913511,3.601937,1422.912929,2462.503698,0.203435,7.459767,0.004844,1.180537,256.216782,2.677455,83.449293,1857.892765,34.934575,10.839938,181.733818,71.93396,675.090909,null
"""std""",null,null,null,null,null,null,null,null,null,null,null,null,null,153.850077,99.73381,2.533364,561.634963,64.60219,7.883975,101.883511,0.301608,4740.351252,14213.846725,4316.041548,16439.995266,4.109926,20.114747,0.256845,142182.796299,1.713125,1.652976,1.155813,278.982599,0.298115,64.728232,66.37034,…,5.588117,4.999781,29.543869,20.954639,3.31196,11.950995,113.743116,1.08125,0.502461,192.543185,5.65566,0.694003,0.265127,16.10081,580.295634,69.690857,479.708876,16.38495,364.919228,462.132109,2.594537,2031.884654,7110.997351,0.73393,13.828313,0.083806,1.534738,704.004027,2.861882,81.735212,4368.261381,29.804947,4.882559,85.662969,55.129898,1434.695314,null
"""min""","""0""","""0""","""0""","""0""","""0""","""0""","""A""","""1""","""2024-03-01 00:10:01.968293+00""","""1""","""0""","""0""","""BS43GGXB""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,117.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,…,0.0,1.0,517.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,13.0,0.0,0.0,10.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,12.0,0.0,0.0,"""test"""
"""25%""",null,null,null,null,null,null,null,null,null,null,null,null,null,100.0,0.0,1.0,0.0,43.0,2.0,0.0,0.0,0.0,2698.0,409.0,1523.0,2.0,24.0,0.0,33059.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,…,2.0,8.0,577.0,16.0,1.0,15.0,27.0,0.0,0.0,95.0,6.0,0.0,0.0,38.0,98.0,16.0,70.0,12.0,92.0,73.0,2.0,371.0,0.0,0.0,1.0,0.0,0.0,93.0,1.0,27.0,0.0,10.0,7.0,119.0,48.0,0.0,null
"""50%""",null,null,null,null,null,null,null,null,null,null,null,null,null,100.0,0.0,2.0,0.0,74.0,5.0,0.0,0.0,0.0,8431.0,843.0,7364.0,3.0,45.0,0.0,113

In [8]:
static.select(cs.all() - cs.string()).describe()

statistic,an,total_balance_on_default_accoun4,seb,total_repayments_on_active_acco2,tic,ef,izb,yeb,as,drb,zd,total_balance_on_active_account1,ueb,hh,tg,em,ah,tp,ej,bn,nic,total_repayments_on_active_acco5,azb,kn,dm,rrb,jg,mr,qeb,yqb,bcc,lrb,fb,gauge2_score,ccc,teb,eb,ye,qic,oic,vm,gsb,ygc,st,tr,hn,wmb,vyb,mu,wm,pob,mhc,pn,sr,pic,jyb,peb,reb,xm,bh,xe,cs,xzb,rh,gg,mzb,fs
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""",4410.0,4542.0,4542.0,4542.0,3997.0,4532.0,3388.0,4542.0,583.0,4526.0,307.0,4542.0,4542.0,1374.0,4542.0,1475.0,4542.0,4542.0,4542.0,4316.0,4542.0,4542.0,4113.0,4295.0,4468.0,1140.0,4538.0,4542.0,4542.0,4542.0,3060.0,4542.0,4542.0,4542.0,2181.0,4542.0,4540.0,4137.0,4542.0,4542.0,4508.0,4101.0,4542.0,4542.0,1278.0,4437.0,4094.0,884.0,727.0,4482.0,4463.0,4542.0,4192.0,1352.0,4542.0,3778.0,4542.0,4542.0,4350.0,4542.0,4033.0,774.0,4509.0,4542.0,4542.0,4255.0,286.0
"""null_count""",132.0,0.0,0.0,0.0,545.0,10.0,1154.0,0.0,3959.0,16.0,4235.0,0.0,0.0,3168.0,0.0,3067.0,0.0,0.0,0.0,226.0,0.0,0.0,429.0,247.0,74.0,3402.0,4.0,0.0,0.0,0.0,1482.0,0.0,0.0,0.0,2361.0,0.0,2.0,405.0,0.0,0.0,34.0,441.0,0.0,0.0,3264.0,105.0,448.0,3658.0,3815.0,60.0,79.0,0.0,350.0,3190.0,0.0,764.0,0.0,0.0,192.0,0.0,509.0,3768.0,33.0,0.0,0.0,287.0,4256.0
"""mean""",122.934467,8.733598,2.353369,260.233377,72.482612,7.119153,10.804014,0.091369,1921.957118,12997.057667,1919.322476,13051.100837,4.245046,41.0,0.045575,138053.352542,1.32915,0.614927,0.369661,161.775023,0.056363,20.749229,3.170921,324.660303,13531.567144,1011.564912,64799.355663,0.541832,0.615148,5.713122,26.156209,5.979304,11.325407,598.522017,34.552499,3.288639,22.905507,114.309161,0.3155,0.117349,111.043035,11.073884,0.27675,0.046455,47.934272,329.372324,61.228627,371.355204,23.887208,145.074743,162.913511,3.601937,1422.912929,2462.503698,0.203435,7.459767,0.004844,1.180537,256.216782,2.677455,83.449293,1857.892765,34.934575,10.839938,181.733818,71.93396,675.090909
"""std""",153.850077,99.73381,2.533364,561.634963,64.60219,7.883975,101.883511,0.301608,4740.351252,14213.846725,4316.041548,16439.995266,4.109926,20.114747,0.256845,142182.796299,1.713125,1.652976,1.155813,278.982599,0.298115,64.728232,66.37034,707.399857,16462.684406,1680.646749,111544.691889,1.423955,1.043987,5.358153,21.028797,5.588117,4.999781,29.543869,20.954639,3.31196,11.950995,113.743116,1.08125,0.502461,192.543185,5.65566,0.694003,0.265127,16.10081,580.295634,69.690857,479.708876,16.38495,364.919228,462.132109,2.594537,2031.884654,7110.997351,0.73393,13.828313,0.083806,1.534738,704.004027,2.861882,81.735212,4368.261381,29.804947,4.882559,85.662969,55.129898,1434.695314
"""min""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,117.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,517.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,13.0,0.0,0.0,10.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,12.0,0.0,0.0
"""25%""",100.0,0.0,1.0,0.0,43.0,2.0,0.0,0.0,0.0,2698.0,409.0,1523.0,2.0,24.0,0.0,33059.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,79.0,1997.0,85.0,7107.0,0.0,0.0,2.0,7.0,2.0,8.0,577.0,16.0,1.0,15.0,27.0,0.0,0.0,95.0,6.0,0.0,0.0,38.0,98.0,16.0,70.0,12.0,92.0,73.0,2.0,371.0,0.0,0.0,1.0,0.0,0.0,93.0,1.0,27.0,0.0,10.0,7.0,119.0,48.0,0.0
"""50%""",100.0,0.0,2.0,0.0,74.0,5.0,0.0,0.0,0.0,8431.0,843.0,7364.0,3.0,45.0,0.0,113397.0,1.0,0.0,0.0,111.0,0.0,0.0,0.0,137.0,8000.0,479.0,19103.0,0.0,0.0,4.0,21.0,4.0,11.0,597.0,36.0,2.0,21.0,75.0,0.0,0.0,98.0,11.0,0.0,0.0,50.0,167.0,41.0,200.0,19.0,98.0,92.0,3.0,866.0,46.0,0.0,2.0,0.0,1.0,103.0,2.0,58.0,0.0,29.0,10.0,174.0,76.0,0.0
"""75%""",115.0,0.0,3.0,281.0,92.0,9.0,0.0,0.0,995.0,18900.0,1964.0,18814.0,6.0,58.0,0.0,193956.0,2.0,0.0,0.0,144.0,0.0,0.0,0.0,292.0,19502.0,1159.0,62181.0,0.0,1.0,8.0,44.0,8.0,14.0,619.0,52.0,4.0,2

In [9]:
ledger = dynamic.join(static, on='bureau_search_ref')

In [10]:
out = {}

for field, dtype in dict(static.schema).items():
    out[field] = "category" if dtype == pl.String else "number"

In [11]:
static['nob'].unique().to_list()

['08',
 '11',
 '07',
 '24',
 '12',
 '19',
 '13',
 '09',
 '22',
 '01',
 '06',
 '18',
 '23',
 '02',
 None,
 '10',
 'XX',
 '15',
 '21',
 '16',
 '04',
 '03',
 '05',
 '17',
 '20',
 '14']

In [12]:
for index, shard in enumerate(ledger.iter_slices(5)):
    shard.write_ndjson(f"../data/lifestreams/plata/plata-{index}.ndjson")

In [13]:
dynamic.select(
    acctypecode=pl.col('bal').list.len()
).get_column('acctypecode').plot.hist()

:Histogram   [acctypecode]   (Count)